In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

2.0.0-beta1


In [2]:
def make_toy_dataset(num_data):
    X = np.random.randn(num_data, 3)
    y = 3 * X[:, 0] - 2 * X[:, 1]**3 + 2 * X[:, 2]**2 + 0.5 * np.random.randn(num_data)
    y = y[:, np.newaxis]
    return X, y

In [3]:
def build_model():
    model = keras.models.Sequential([
        keras.layers.Dense(10, activation="relu", input_dim=3),
        keras.layers.Dense(10, activation="relu"),
        keras.layers.Dense(1)
    ])

    return model

In [4]:
N_train = 1000
N_test = 200

num_epochs = 25
learning_rate = 0.01
batch_size = 64

In [5]:
# get toy datasets
X_train, y_train = make_toy_dataset(N_train)
X_test, y_test = make_toy_dataset(N_test)

# make datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=512).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(100)

In [6]:
model = build_model()
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
loss_fn = keras.losses.MeanSquaredError()

train_loss = keras.metrics.Mean()
test_loss = keras.metrics.Mean()
train_mae_metric = keras.metrics.MeanAbsoluteError()
test_mae_metric = keras.metrics.MeanAbsoluteError()


for epoch in range(1, num_epochs+1):
    
    # train step 
    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            y_pred = model(x_batch)
            loss = loss_fn(y_batch, y_pred)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # accumulate metrics of one batch
        train_loss(loss)
        train_mae_metric(y_batch, y_pred)
    
    
    # get metrics of one epoch and reset their states
    train_loss_result = train_loss.result()
    train_loss.reset_states()
    
    train_mae_result = train_mae_metric.result()
    train_mae_metric.reset_states()
    
    
    # evaluation step
    for x_batch, y_batch in test_dataset:
        y_pred = model(x_batch)
        loss = loss_fn(y_batch, y_pred)
        
        # accumulate metrics
        test_loss(loss)
        test_mae_metric(y_batch, y_pred)
    
    
    # get metrics and reset their states
    test_loss_result = test_loss.result()
    test_loss.reset_states()
    
    test_mae_result = test_mae_metric.result()
    test_mae_metric.reset_states()
    
    
    # print train logs
    template = "epoch {}: loss={:.3f}, val_loss={:3f}, mae={:.3f}, val_mae={:.3f}"
    print(template.format(epoch, train_loss_result, test_loss_result, train_mae_result, test_mae_result))

epoch 1: loss=73.471, val_loss=29.793621, mae=4.619, val_mae=3.468
epoch 2: loss=39.127, val_loss=28.593746, mae=3.485, val_mae=3.690
epoch 3: loss=32.429, val_loss=24.075203, mae=3.167, val_mae=3.273
epoch 4: loss=26.503, val_loss=20.386633, mae=2.782, val_mae=2.959
epoch 5: loss=21.783, val_loss=16.858789, mae=2.495, val_mae=2.623
epoch 6: loss=17.787, val_loss=13.565987, mae=2.221, val_mae=2.188
epoch 7: loss=14.618, val_loss=12.462762, mae=1.948, val_mae=2.182
epoch 8: loss=12.315, val_loss=11.677664, mae=1.741, val_mae=2.187
epoch 9: loss=10.510, val_loss=10.945240, mae=1.603, val_mae=2.165
epoch 10: loss=9.509, val_loss=10.479336, mae=1.547, val_mae=2.178
epoch 11: loss=8.434, val_loss=9.961853, mae=1.474, val_mae=2.111
epoch 12: loss=7.672, val_loss=9.180765, mae=1.437, val_mae=2.052
epoch 13: loss=6.965, val_loss=8.860435, mae=1.398, val_mae=1.976
epoch 14: loss=6.578, val_loss=9.045691, mae=1.379, val_mae=1.994
epoch 15: loss=6.204, val_loss=7.967905, mae=1.351, val_mae=1.869
